## Convolutional Neural Networks using PyTorch

### 0. Importing the neccessary packages

In [2]:
# Basic packages
import numpy as np
import pandas as pd
import os
import cv2

# PyTorch related packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms

#Data Split
from sklearn.model_selection import train_test_split

### 1. Basic Data Investigation

#### - Defining the data path

In [2]:
train_path = '../input/train/train/'
test_path = '../input/test/test/'
labels = pd.read_csv('../input/train.csv')
sub = pd.read_csv('../input/sample_submission.csv')

In [4]:
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


#### - Inspecting the target balance

In [3]:
print('Num train samples:{0}'.format(len(os.listdir(train_path))))
print('Num test samples:{0}'.format(len(os.listdir(test_path))))

Num train samples:17501
Num test samples:4000


##### The target has a 3:1 proportion, but will not be doing any balancing 

In [5]:
labels['has_cactus'].value_counts()

1    13136
0     4364
Name: has_cactus, dtype: int64

### 2. Convolutional Neural Network on Pytorch

#### - Parameters for model

In [10]:
# Hyper parameters
num_epochs = 25
num_classes = 2
batch_size = 128
learning_rate = 0.002

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#### - Data Splitting

##### The dataset was split into train and test by 9:1

In [11]:
# data splitting
train, val = train_test_split(labels, stratify=labels.has_cactus, test_size=0.3)
train.shape, val.shape

((15750, 2), (1750, 2))

In [12]:
train['has_cactus'].value_counts()

1    11822
0     3928
Name: has_cactus, dtype: int64

In [13]:
val['has_cactus'].value_counts()

1    1314
0     436
Name: has_cactus, dtype: int64

### 3. Simple Custom Generator

In [14]:
class Cactus(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name)
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

#### - Image Preprocessing

In [15]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(32, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(32, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

In [ ]:
# Data generators
dataset_train = Cactus(df_data=train, 
                          data_dir=train_path, 
                          transform=trans_train)

dataset_valid = Cactus(df_data=val, 
                          data_dir=train_path, 
                          transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, 
                          batch_size=batch_size, 
                          shuffle=True,
                          num_workers=0)

loader_valid = DataLoader(dataset = dataset_valid,
                          batch_size=batch_size//2, 
                          shuffle=False, 
                          num_workers=0)

### 4. Model

In [16]:
# NOTE: class is inherited from nn.Module
class CNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(model, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(4)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
   
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        #print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [17]:
model = CNN().to(device)

In [18]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

#### - Train the model

In [19]:
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/25], Step [100/124], Loss: 0.0465
Epoch [2/25], Step [100/124], Loss: 0.0405
Epoch [3/25], Step [100/124], Loss: 0.0323
Epoch [4/25], Step [100/124], Loss: 0.0755
Epoch [5/25], Step [100/124], Loss: 0.0017
Epoch [6/25], Step [100/124], Loss: 0.0033
Epoch [7/25], Step [100/124], Loss: 0.0300
Epoch [8/25], Step [100/124], Loss: 0.0305
Epoch [9/25], Step [100/124], Loss: 0.0124
Epoch [10/25], Step [100/124], Loss: 0.0029
Epoch [11/25], Step [100/124], Loss: 0.0264
Epoch [12/25], Step [100/124], Loss: 0.0007
Epoch [13/25], Step [100/124], Loss: 0.0013
Epoch [14/25], Step [100/124], Loss: 0.0123
Epoch [15/25], Step [100/124], Loss: 0.0060
Epoch [16/25], Step [100/124], Loss: 0.0183
Epoch [17/25], Step [100/124], Loss: 0.0012
Epoch [18/25], Step [100/124], Loss: 0.0011
Epoch [19/25], Step [100/124], Loss: 0.0291
Epoch [20/25], Step [100/124], Loss: 0.0011
Epoch [21/25], Step [100/124], Loss: 0.0011
Epoch [22/25], Step [100/124], Loss: 0.0031
Epoch [23/25], Step [100/124], Loss: 0.00

### 4. Accuracy Check

In [20]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model on the 1750 validation images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 1750 validation images: 99.48571428571428 %
